In [54]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
# import pyspark
import json
import ast
import boto3
from sagemaker import get_execution_role
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.decomposition import TruncatedSVD

from IPython.display import clear_output

TRANSACTION DATA PREP

In [55]:
role = get_execution_role()
bucket='sagemaker-us-east-1-770526846351'

In [56]:
df_tax_location = 's3://{}/{}'.format(bucket, 'tax_investor_filtered.csv')
df = pd.read_csv(df_tax_location)

In [57]:
df = df[df['tax_amount'].notna()]
df = df[df['value_land'].notna()]

In [58]:
df['taxRatio'] = df['tax_amount']/df['purchasePrice']
df['landRatio'] = df['value_land']/df['value']
df['impRatio'] = 1 - df['landRatio']

In [59]:
df_txn = df[['id', 'sqft', 'value', 'purchasePrice', 'propertyType', 'pricePerSqft', 
        'valuePerSqft', 'zip', 'state', 'taxRatio', 'landRatio', 'impRatio']]

In [60]:
# create dummy vars

dummies = ['state', 'propertyType']


for col in dummies:
    vals = df_txn[col].unique().tolist()
    df_txn[col] = pd.Categorical(df_txn[col], categories=vals)
    

df_txn = pd.get_dummies(df_txn, columns=dummies, drop_first=True)


clear_output()

In [61]:
# set index

df_txn = df_txn.set_index('id')

In [62]:
# scale inputs

scaler_prop = StandardScaler()
txn_scaled = scaler_prop.fit_transform(df_txn)

In [63]:
# dimensionality reduction

txn_pca = PCA(n_components=10)
txn_scaled_pca = txn_pca.fit_transform(txn_scaled)

In [64]:
print(txn_pca.explained_variance_ratio_)

[0.22008151 0.16745542 0.1445849  0.11632359 0.07894299 0.07690785
 0.07524492 0.07353989 0.02744695 0.0120335 ]


In [65]:
sum(txn_pca.explained_variance_ratio_)

0.9925615085940344

In [66]:
# Final df

df_txn = pd.DataFrame(txn_scaled_pca, index=df_txn.index) #columns=df_txn.columns

In [67]:
df_txn.head()

,0,1,2,3,4,5,6,7,8,9
id,,,,,,,,,,
11001-20220713-192-0,1.915286,1.129846,0.342576,0.988148,0.251207,-0.027855,-0.363346,0.126845,-1.511220,-0.760808
11001-20210629-297-0,1.698778,1.079330,-0.266098,0.260618,0.340104,-0.033733,-0.206122,0.215363,-1.263419,-1.054342
11001-20220826-208-1,1.631475,1.151922,-0.344449,-0.134238,0.444768,-0.039329,-0.132546,0.281944,-1.337601,-1.098229
11001-20190221-139-1,1.650858,-0.318679,0.651853,1.774723,0.315576,-0.027269,-0.445488,0.140650,-1.808860,-0.368423
11001-20220325-201-1,2.016636,1.021095,0.517227,1.588420,0.509905,-0.037716,-0.500223,0.312668,-1.957268,-0.380496


In [68]:
df_txn.shape

(24054, 10)

In [69]:
# df_txn.to_csv('s3://{}/{}'.format(bucket, 'region_transaction_sim_input.csv'))

#### Cosine Similarity

In [70]:
txn_sim_matrix = cosine_similarity(df_txn)

In [71]:
txn_sim = pd.DataFrame(txn_sim_matrix, columns=df_txn.index, index=df_txn.index)

In [72]:
txn_sim.head()

id,11001-20220713-192-0,11001-20210629-297-0,11001-20220826-208-1,11001-20190221-139-1,11001-20220325-201-1,11001-20190822-116-0,11001-20181009-61-1,11001-20210119-468-1,11001-20211104-41-1,11001-20180705-225-1,...,51169-20220304-37-0,51059-20220126-836-0,51550-20210130-500-0,51059-20220610-39-0,51169-20190222-32-0,51169-20221125-636-0,51550-20210501-821-1,51550-20220606-428-0,51169-20220114-19-0,51169-20220603-33-1
id,,,,,,,,,,,,,,,,,,,,,
11001-20220713-192-0,1.000000,0.937097,0.879314,0.838173,0.971314,0.745133,0.744073,0.916528,0.785865,0.810391,...,-0.597308,0.471770,-0.130093,0.330910,-0.540700,-0.360610,-0.417040,-0.268567,-0.772270,-0.827964
11001-20210629-297-0,0.937097,1.000000,0.986449,0.666583,0.852000,0.550930,0.560894,0.744993,0.568453,0.662371,...,-0.469827,0.487630,-0.093688,0.377563,-0.406683,-0.251323,-0.349399,-0.178235,-0.670686,-0.732393
11001-20220826-208-1,0.879314,0.986449,1.000000,0.568563,0.778714,0.474874,0.493390,0.648750,0.454013,0.591279,...,-0.371517,0.495002,-0.042405,0.406802,-0.312632,-0.182254,-0.298750,-0.104387,-0.588465,-0.640373
11001-20190221-139-1,0.838173,0.666583,0.568563,1.000000,0.913247,0.954706,0.942340,0.975447,0.986830,0.976856,...,-0.785429,0.546858,-0.354304,0.405247,-0.770437,-0.658540,-0.589313,-0.505603,-0.856681,-0.830527
11001-20220325-201-1,0.971314,0.852000,0.778714,0.913247,1.000000,0.857673,0.853081,0.973152,0.879613,0.893603,...,-0.620472,0.454504,-0.121797,0.295831,-0.573511,-0.397356,-0.434260,-0.286355,-0.787001,-0.825737


In [73]:
txn_result = {'id':[], 'recs':[]}

for idx, row in txn_sim.iterrows():
    data = txn_sim[idx].sort_values(ascending=False).reset_index(name='cosine_score').head(11).tail(10).to_dict('records')
    txn_result['id'].append(idx)
    txn_result['recs'].append(data)

In [74]:
df_txn_result = pd.DataFrame(txn_result)

In [75]:
df_txn_result.head()

,id,recs
0,11001-20220713-192-0,"[{'id': '11001-20220912-55-0', 'cosine_score':..."
1,11001-20210629-297-0,"[{'id': '11001-20220819-72-1', 'cosine_score':..."
2,11001-20220826-208-1,"[{'id': '11001-20200713-547-0', 'cosine_score'..."
3,11001-20190221-139-1,"[{'id': '11001-20220824-106-1', 'cosine_score'..."
4,11001-20220325-201-1,"[{'id': '11001-20211220-253-1', 'cosine_score'..."


In [77]:
df_txn_result.shape

(24054, 2)

In [78]:
df_txn_result.to_csv('s3://{}/{}'.format(bucket, 'region_transaction_cosine_sim_result_top10.csv'))

#### Euclidean Distance

In [79]:
txn_eu_sim_matrix = euclidean_distances(df_txn)

In [80]:
txn_eu_sim = pd.DataFrame(txn_eu_sim_matrix, columns=df_txn.index, index=df_txn.index)

In [81]:
txn_eu_sim.head()

id,11001-20220713-192-0,11001-20210629-297-0,11001-20220826-208-1,11001-20190221-139-1,11001-20220325-201-1,11001-20190822-116-0,11001-20181009-61-1,11001-20210119-468-1,11001-20211104-41-1,11001-20180705-225-1,...,51169-20220304-37-0,51059-20220126-836-0,51550-20210130-500-0,51059-20220610-39-0,51169-20190222-32-0,51169-20221125-636-0,51550-20210501-821-1,51550-20220606-428-0,51169-20220114-19-0,51169-20220603-33-1
id,,,,,,,,,,,,,,,,,,,,,
11001-20220713-192-0,0.000000,1.066429,1.439343,1.770914,0.936382,2.709047,2.660946,1.424946,2.351794,1.914146,...,4.849819,2.799900,3.646776,3.169590,4.795826,4.544970,3.931351,3.841130,5.035889,5.263213
11001-20210629-297-0,1.066429,0.000000,0.447337,2.433801,1.881857,3.416797,3.324779,2.350781,3.168819,2.446069,...,4.343588,2.529920,3.287122,2.828638,4.279944,4.070635,3.524903,3.406367,4.563001,4.792811
11001-20220826-208-1,1.439343,0.447337,0.000000,2.772945,2.216980,3.662028,3.545695,2.716862,3.531835,2.698635,...,4.245864,2.547958,3.265785,2.798153,4.183225,4.002991,3.517503,3.356962,4.502816,4.717193
11001-20190221-139-1,1.770914,2.433801,2.772945,0.000000,1.440226,1.394551,1.449377,0.816695,0.838784,0.684259,...,5.279830,2.721067,4.098521,3.109991,5.292199,5.163790,4.288253,4.298967,5.311428,5.424878
11001-20220325-201-1,0.936382,1.881857,2.216980,1.440226,0.000000,2.089270,2.082188,0.817692,1.818718,1.586149,...,5.362855,3.221458,4.098549,3.632741,5.319051,5.058904,4.436771,4.336971,5.551770,5.757692


In [82]:
txn_eu_result = {'id':[], 'recs':[]}

for idx, row in txn_eu_sim.iterrows():
    data = txn_eu_sim[idx].sort_values(ascending=True).reset_index(name='euclidean_distance').head(11).tail(10).to_dict('records')
    txn_eu_result['id'].append(idx)
    txn_eu_result['recs'].append(data)

In [83]:
df_txn_eu_result = pd.DataFrame(txn_eu_result)

In [84]:
df_txn_eu_result.head()

,id,recs
0,11001-20220713-192-0,"[{'id': '11001-20220912-55-0', 'euclidean_dist..."
1,11001-20210629-297-0,"[{'id': '11001-20220819-72-1', 'euclidean_dist..."
2,11001-20220826-208-1,"[{'id': '11001-20200713-547-0', 'euclidean_dis..."
3,11001-20190221-139-1,"[{'id': '11001-20220824-106-1', 'euclidean_dis..."
4,11001-20220325-201-1,"[{'id': '11001-20211220-253-1', 'euclidean_dis..."


In [85]:
df_txn_eu_result.to_csv('s3://{}/{}'.format(bucket, 'region_transaction_euclidean_sim_result_top10.csv'))